## Redis Assignment Details
In this assignment, you will access a Redis server using provided credentials and perform various queries to retrieve information.

In [ ]:

import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    !pip install redis
import redis


rd = redis.Redis(host='lab.aimet.tech', charset="utf-8", decode_responses=True)
rd.auth(username='hw', password='hw')

### Query 1: Retrieve a Username
Retrieve the username associated with user id '600'.

In [ ]:
rd.get('user:600:name')

### Query 2: Retrieve a User ID
Find the user ID associated with the username 'excitedPie4'.

In [ ]:

id_excitedPie4 = rd.get('username:excitedPie4')
id_excitedPie4


### Query 3: Count Following
Determine how many users 'excitedPie4' is following.

In [ ]:

rd.smembers(f'user:{id_excitedPie4}:follows')
rd.scard(f'user:{id_excitedPie4}:follows')


### Query 4: Total Users
Count the total number of users in the database using the SCAN command.

In [ ]:

def scan(rd, match):
    cursor, keys = 0, []
    while True:
        cursor, partial_keys = rd.scan(cursor=cursor, match=match)
        keys.extend(partial_keys)
        if cursor == 0:
            break
    return keys

all_users = scan(rd, 'username:*')
print(all_users)
number_of_users = len(all_users)
print(f"Number of users in the database: {number_of_users}")


### Analysis: Average Follows
Calculate the average number of follows per user.

In [ ]:

import numpy as np
follows = scan(rd, 'user:*:follows')
follows_num = [rd.scard(follow) for follow in follows]
np.sum(follows_num) / number_of_users


### Analysis: Users Following 5-10 Users
Identify how many users follow between 5 and 10 other users.

In [ ]:

filtered_follows_num = np.array(follows_num)[(np.array(follows_num) >= 5) & (np.array(follows_num) <= 10)]
filtered_follows_num.shape[0]


### Analysis: Most Followers
Determine which user account has the most followers.

In [ ]:

user_follower = scan(rd, 'user:*:followed_by')
followers_num = [rd.scard(follower) for follower in user_follower]
user_follower[np.argmax(np.array(followers_num))]
